In [67]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate   
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [68]:
train_data = pd.read_csv('C:\\Users\\w9012219\\Desktop\\Python\\0909 Esun\\total_set_1022.csv')
train_data=train_data[train_data['if_train']==1]

# Feature selection

In [69]:
#---feature columns----#
#(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
#       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'if_train', 'insfg',
#       'iterm', 'locdt', 'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn',
#       'stscd', 'txkey', 'time_interval', 'if_mor', 'conam_log', 'conam_3rt',
#       
#       'mchno1', 'mchno2', 'mchno3', 'conam1', 'conam2', 'conam3', 'mcc1',
#       'mcc2', 'mcc3','scity1', 'scity2', 'scity3', 'acqic1', 'acqic2',
#       'acqic3', 'contp1', 'contp2', 'contp3', 'ovrlt1', 'ovrlt2', 'ovrlt3',
#       'hcefg1', 'hcefg2', 'hcefg3', 'csmcu1', 'csmcu2', 'csmcu3', 'if_mor1',
#       'if_mor2', 'if_mor3', 'conam_log1', 'conam_log2', 'conam_log3',
#       'conam_3rt1', 'conam_3rt2', 'conam_3rt3', 
#      
#       'weekday', 'time_weight',
#       'sum_lag0', 'sum_lag1', 'sum_lag2', 'diff_log10', 'mcc_mode'])

feat_col = ['acqic','conam','contp','csmcu', 'etymd',
            'hcefg', 'scity','stocn',
            'iterm','loctm','mcc','mchno', 
            'stscd','time_interval','if_mor','conam_log']

feat_col.extend(lagged_cate)
feat_col.extend(lagged_num)
feat_col.extend(['weekday','time_weight'])

print(feat_col)

['acqic', 'conam', 'contp', 'csmcu', 'etymd', 'hcefg', 'scity', 'stocn', 'iterm', 'loctm', 'mcc', 'mchno', 'stscd', 'time_interval', 'if_mor', 'conam_log', 'mchno1', 'mchno2', 'mchno3', 'mcc1', 'mcc2', 'mcc3', 'scity1', 'scity2', 'scity3', 'acqic1', 'acqic2', 'acqic3', 'if_mor1', 'if_mor2', 'if_mor3', 'conam1', 'conam2', 'conam3', 'conam_log1', 'conam_log2', 'conam_log3', 'weekday', 'time_weight']


In [70]:
train_data.columns

Index(['acqic', 'acqic1', 'acqic2', 'acqic3', 'bacno', 'cano', 'conam',
       'conam1', 'conam2', 'conam3', 'conam_3rt', 'conam_3rt1', 'conam_3rt2',
       'conam_3rt3', 'conam_log', 'conam_log1', 'conam_log2', 'conam_log3',
       'contp', 'contp1', 'contp2', 'contp3', 'csmcu', 'csmcu1', 'csmcu2',
       'csmcu3', 'diff_log10', 'ecfg', 'etymd', 'flbmk', 'flg_3dsmk',
       'fraud_ind', 'hcefg', 'hcefg1', 'hcefg2', 'hcefg3', 'if_mor', 'if_mor1',
       'if_mor2', 'if_mor3', 'if_train', 'insfg', 'iterm', 'locdt', 'loctm',
       'mcc', 'mcc1', 'mcc2', 'mcc3', 'mcc_mode', 'mchno', 'mchno1', 'mchno2',
       'mchno3', 'ovrlt', 'ovrlt1', 'ovrlt2', 'ovrlt3', 'scity', 'scity1',
       'scity2', 'scity3', 'stocn', 'stscd', 'sum_lag0', 'sum_lag1',
       'sum_lag2', 'time_interval', 'time_weight', 'txkey', 'weekday'],
      dtype='object')

In [71]:
train_data_y = train_data['fraud_ind']

In [72]:
train_data = train_data[feat_col]

## lag數值特徵na補0
train_data[feat_col] = train_data[feat_col].fillna(0)


In [73]:
train_data['fraud_ind']=train_data_y.astype(int).astype(str)

In [74]:
train_data.head(3)

,acqic,conam,contp,csmcu,etymd,hcefg,scity,stocn,iterm,loctm,...,if_mor3,conam1,conam2,conam3,conam_log1,conam_log2,conam_log3,weekday,time_weight,fraud_ind
0,6032,236.43,5,62,4,5,5817,102,0,151948.0,...,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,1,0.03125,0
1,6032,319.72,5,62,4,5,5817,102,0,154407.0,...,0.0,236.43,0.00,0.0,5.469873,0.000000,0.0,4,0.03125,0
2,6716,1016.34,5,62,4,5,5820,102,0,145342.0,...,0.0,319.72,236.43,0.0,5.770568,5.469873,0.0,6,0.03125,0


# Split train/validation set

In [75]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(train_data[feat_col] ,
                                               train_data['fraud_ind'],
                                              stratify = train_data['fraud_ind'],
                                              test_size = 0.2)

# GridSearch for XGBoost 

In [76]:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate   
from sklearn.metrics import roc_auc_score

In [77]:
parameters = {'max_depth':[3,5,7,8],
              #'max_delta_step': [0, 0.6, 1, 2],
              #'colsample_bytree': [ 0.8, 0.9],
              #'reg_alpha': [0,  0.5,  1],
              #'reg_lambda': [0.2,  0.6,  1],
              #'scale_pos_weight': [0.2, 0.6, 1]

}

In [78]:
xlf = xgb.XGBClassifier(
			learning_rate=0.03,
			n_estimators=500, 
			silent=True,
			objective='binary:logistic',
			nthread=-1,
			gamma=0,
			min_child_weight=0,
			subsample=0.85,
			colsample_bytree=0.7,
			colsample_bylevel=1,
			scoring='roc_auc',                  
			seed=1440,
			missing=None)

In [79]:
# 有了gridsearch我们便不需要fit函数
gsearch = GridSearchCV(xlf, param_grid=parameters, scoring='roc_auc', cv=3)
gsearch.fit(x_val, y_val)

print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.979
Best parameters set:
	max_depth: 8
